### Problem 1
#### Question 2 : Hyperparameter search

In [16]:
# USEFUL IMPORTS
import mlp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import sklearn.model_selection
import time
import math
import gc

In [10]:
# LOAD THE DATA
data = pd.read_csv("../../mnist_train.csv")
data2 = pd.read_csv("../../mnist_test.csv")

In [11]:
# SPLIT THEM INTO TRAIN VALID TEST SETS
X = data.values[:,1:]/255
Y = data.values[:,0]
X_test = data2.values[:, 1:]/255
Y_test = data2.values[:, 0]
X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split( X, Y, test_size=0.1666666, random_state=42)

In [12]:
print("Train set shape ", X_train.shape)
print("Valid set shape ", X_valid.shape)
print("Test  set shape ", X_test.shape)

Train set shape  (50000, 784)
Valid set shape  (10000, 784)
Test  set shape  (10000, 784)


#### Time test

In [13]:
start = time.time()
model = mlp.MLP_2L(784, 784, 480, 10, init="glorot")
model.fit(X_train, Y_train, 1, 1, 0.001, validation_data=(X_valid, Y_valid), epsilon=0.05)
end = time.time()
duration = end - start

days = int(duration/(24 * 60 * 60))
duration = duration % (24 * 60 * 60)
hours = int(duration/( 60 * 60))
duration = duration % (60 * 60)
minutes = int(duration/60)

duration2 = 4 * 128 * duration
days2 = int(duration2/(24 * 60 * 60))
duration2 = duration2 % (24 * 60 * 60)
hours2 = int(duration2/( 60 * 60))
duration2 = duration2 % (60 * 60)
minutes2 = int(duration2/60)
print("One training epoch with {:d} samples using SGD takes {:d}d {:2d}h {:2d}m".format(X_train.shape[0],days, hours, minutes ))
print("Then the whole fine tuning should take {:d}d {:2d}h {:2d}m".format(days2, hours2, minutes2 ))

Input dimension 784	Layer 1 dimension 784	Layer 2 dimension 480	Output dimension 10	 Initilization method glorot	Activation function identity
Total number of parameters : 997050
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/1		Total training time 0.0s
	Samples 50000/50000	Epoch time 920.65s	Accuracy 0.913	Loss 0.306	Valid accuracy 0.910	 Valid loss 0.309

Total training time 920.65s
One training epoch with 50000 samples using SGD takes 0d  0h 15m
Then the whole fine tuning should take 13d 15h 20m


##### Fine tuning the hyperparameters

In [18]:
train_start = time.time()
input_size = 784
output_size = 10
count = 0
models_perf = []
# print("{:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s}".format("hidden 1", "hidden 2", "activation" , "l_rate", "l1", "l2",  "accuracy"))
for n1 in [ 500, 784]:
    for n2 in [300, 480]:
        n_parameter = n1 * (input_size + 1) + n2 * (n1 + 1) + output_size * (n2 + 1)
        if n_parameter > 1e6 or n_parameter < 5e5: continue
        for activation in ["identity", "relu", "sigmoid", "tanh"]:
            for learning_rate in [0.0001, 0.001 ]:
                for l1 in [ 1e-5]:
                    for l2 in [ 1e-5]:
                        count +=1
                        
                        print("Model {:d}/128".format(count))
                        model = mlp.MLP_2L(input_size, n1, n2, output_size, init="glorot", verbose=False)
                        
                        model.fit(X_train, Y_train, 2, 10, learning_rate, validation_data=(X_valid, Y_valid), epsilon=0.001 )
                        model.fit(X_train, Y_train, 3, 1, learning_rate, validation_data=(X_valid, Y_valid), epsilon=0.001 )
                        model.save("models/model"+str(count))
                        perf = dict()
                        perf["n1"] = n1
                        perf["n2"] = n2
                        perf["activation"] = activation
                        perf["learning_rate"] = learning_rate
                        perf["l1"] = l1
                        perf["l2"] = l2
                        perf["accuracy"] = model.evaluate(X_valid, Y_valid)
                        
                        models_perf.append(perf)
                        del model
                        gc.collect()
train_end = time.time()
duration3 = train_end - train_start
days3 = int(duration3/(24 * 60 * 60))
duration3 = duration3 % (24 * 60 * 60)
hours3 = int(duration3/( 60 * 60))
duration3 = duration3 % (60 * 60)
minutes3 = int(duration3/60)

print("Then the whole fine tuning took {:d}d {:2d}h {:2d}m".format(days3, hours3, minutes3 ))

Model 1/128
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/2		Total training time 0.0s
	Samples 50000/50000	Epoch time 55.83s	Accuracy 0.745	Loss 1.138	Valid accuracy 0.744	 Valid loss 1.139
Epoch 2/2		Total training time 55.8s
	Samples 50000/50000	Epoch time 49.43s	Accuracy 0.810	Loss 0.818	Valid accuracy 0.808	 Valid loss 0.819

Total training time 105.27s
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/3		Total training time 0.0s
	Samples 50000/50000	Epoch time 451.93s	Accuracy 0.885	Loss 0.415	Valid accuracy 0.884	 Valid loss 0.412
Epoch 2/3		Total training time 451.9s
	Samples 50000/50000	Epoch time 464.43s	Accuracy 0.898	Loss 0.360	Valid accuracy 0.898	 Valid loss 0.357
Epoch 3/3		Total training time 916.4s
	Samples 50000/50000	Epoch time 477.96s	Accuracy 0.905	Loss 0.336	Valid accuracy 0.904	 Valid loss 0.333

Total training time 1394.32s
Model 2/128
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/2		Total training time 0.0s
	Samples 50000/50000	Ep

	Samples 50000/50000	Epoch time 60.37s	Accuracy 0.897	Loss 0.367	Valid accuracy 0.898	 Valid loss 0.361

Total training time 123.03s
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/3		Total training time 0.0s
	Samples 50000/50000	Epoch time 519.53s	Accuracy 0.914	Loss 0.301	Valid accuracy 0.913	 Valid loss 0.305
Epoch 2/3		Total training time 519.5s
	Samples 50000/50000	Epoch time 517.23s	Accuracy 0.919	Loss 0.287	Valid accuracy 0.916	 Valid loss 0.295
Epoch 3/3		Total training time 1036.8s
	Samples 50000/50000	Epoch time 520.87s	Accuracy 0.922	Loss 0.279	Valid accuracy 0.918	 Valid loss 0.291

Total training time 1557.62s
Model 11/128
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/2		Total training time 0.0s
	Samples 50000/50000	Epoch time 60.58s	Accuracy 0.756	Loss 1.166	Valid accuracy 0.753	 Valid loss 1.168
Epoch 2/2		Total training time 60.6s
	Samples 50000/50000	Epoch time 60.37s	Accuracy 0.813	Loss 0.825	Valid accuracy 0.811	 Valid loss 0.827

Total training

	Samples 50000/50000	Epoch time 668.23s	Accuracy 0.888	Loss 0.405	Valid accuracy 0.887	 Valid loss 0.403
Epoch 2/3		Total training time 668.2s
	Samples 50000/50000	Epoch time 667.42s	Accuracy 0.900	Loss 0.355	Valid accuracy 0.901	 Valid loss 0.353
Epoch 3/3		Total training time 1335.6s
	Samples 50000/50000	Epoch time 751.88s	Accuracy 0.906	Loss 0.333	Valid accuracy 0.906	 Valid loss 0.331

Total training time 2087.53s
Model 20/128
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/2		Total training time 0.0s


KeyboardInterrupt: 

#### Results

In [60]:
print("{:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s}".format("hidden 1", "hidden 2", "activation" , "l_rate", "l1", "l2",  "accuracy"))
print("________________________________________________________________________________________________")
for perf in models_perf:
    print("{:^10d} | {:^10d} | {:^10s} | {:^10f} | {:^10f} | {:^10f} | {:^10f}".format(perf["n1"], perf["n2"] , perf["activation"], perf["learning_rate"], perf["l1"], perf["l2"], perf["accuracy"]))

One training epoch with 10 samples using SGD takes 0d  6h 30m
